In [50]:
# pip3 install pandas os numpy scikit-learn matplotlib seaborn torch 

In [51]:
import pandas as pd
import os
import numpy as np
import shutil

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from PIL import Image

In [52]:
metadata = pd.read_csv("data/gz2sample.csv.gz", compression="gzip")
metadata.to_csv("data/gz2sample.csv")
target_column_df = pd.read_csv("data/gz2_hart16.csv")

target column

In [53]:
target_col = target_column_df["t04_spiral_a08_spiral_flag"]

In [54]:
target_col.head()

0    1
1    1
2    0
3    1
4    0
Name: t04_spiral_a08_spiral_flag, dtype: int64

In [55]:
target_col.describe()

count    3020.000000
mean        0.347682
std         0.476313
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: t04_spiral_a08_spiral_flag, dtype: float64

In [56]:
target_col.info()

<class 'pandas.core.series.Series'>
RangeIndex: 3020 entries, 0 to 3019
Series name: t04_spiral_a08_spiral_flag
Non-Null Count  Dtype
--------------  -----
3020 non-null   int64
dtypes: int64(1)
memory usage: 23.7 KB


In [57]:
target_col.shape

(3020,)

In [58]:
null_target = target_col.isnull().sum()
print(null_target)

0


In [59]:
flag_distribution = target_col.value_counts()
print(flag_distribution)

t04_spiral_a08_spiral_flag
0    1970
1    1050
Name: count, dtype: int64


metadata

In [60]:
metadata.head()

,OBJID,RUN,RERUN,CAMCOL,FIELD,OBJ,RA,DEC,PETROR50_R,PETROR90_R,...,PETROMAGERR_MG,PETROMAGERR_MR,PETROMAGERR_MI,PETROMAGERR_MZ,PETROR50_R_KPC,REGION,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN,WVT_BIN
0,587722981736054938,752,40,1,182,154,170.805579,-1.182146,4.486662,10.670979,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
1,587722981736120347,752,40,1,183,27,170.945892,-1.108325,13.407897,26.012415,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
2,587722981736579107,752,40,1,190,35,171.990936,-1.214009,8.175695,15.858483,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
3,587722981739069541,752,40,1,228,101,177.631858,-1.105126,2.929065,8.661545,...,NaN,NaN,NaN,NaN,NaN,2,-9999,-9999,-9999,0
4,587722981741363294,752,40,1,263,94,182.925262,-1.092357,2.695631,7.912694,...,0.016804,0.013804,0.014019,0.01994,3.973422,2,39,50,6,66


In [61]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325704 entries, 0 to 325703
Data columns (total 63 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   OBJID                      325704 non-null  int64  
 1   RUN                        325704 non-null  int64  
 2   RERUN                      325704 non-null  int64  
 3   CAMCOL                     325704 non-null  int64  
 4   FIELD                      325704 non-null  int64  
 5   OBJ                        325704 non-null  int64  
 6   RA                         325704 non-null  float64
 7   DEC                        325704 non-null  float64
 8   PETROR50_R                 325704 non-null  float64
 9   PETROR90_R                 325704 non-null  float64
 10  PETROMAG_U                 325704 non-null  float64
 11  PETROMAG_G                 325704 non-null  float64
 12  PETROMAG_R                 325704 non-null  float64
 13  PETROMAG_I                 32

In [62]:
metadata.describe()

,OBJID,RUN,RERUN,CAMCOL,FIELD,OBJ,RA,DEC,PETROR50_R,PETROR90_R,...,PETROMAGERR_MG,PETROMAGERR_MR,PETROMAGERR_MI,PETROMAGERR_MZ,PETROR50_R_KPC,REGION,PETROR50_R_KPC_SIMPLE_BIN,PETROMAG_MR_SIMPLE_BIN,REDSHIFT_SIMPLE_BIN,WVT_BIN
count,3.257040e+05,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000,...,272542.000000,272542.000000,272542.000000,272542.000000,272542.000000,325704.000000,325704.000000,325704.000000,325704.000000,325704.000000
mean,1.340051e+18,3237.238486,36.718005,3.490009,217.833085,192.306803,184.034903,22.991662,3.249040,8.457078,...,3.702066,0.395241,6.631378,33.427876,4.311567,3.846348,-1594.583613,-1585.788299,-1576.455776,30.976681
std,2.344549e+18,1556.824795,11.155879,1.540730,164.813612,156.626991,72.178687,19.682874,24.874690,25.218483,...,605.719636,191.548567,812.646244,1826.954566,37.255247,2.679289,3712.106273,3716.071712,3728.635743,30.246290
min,5.877230e+17,106.000000,2.000000,1.000000,10.000000,1.000000,0.003800,-3.743851,-9999.000000,-9999.000000,...,0.010009,0.010008,0.010010,0.010050,-19408.213000,0.000000,-9999.000000,-9999.000000,-9999.000000,0.000000
25%,5.877328e+17,2243.000000,40.000000,2.000000,98.000000,105.000000,147.316952,3.798398,2.151452,5.917597,...,0.013134,0.012251,0.012697,0.019823,2.781042,2.000000,19.000000,28.000000,2.000000,2.000000
50%,5.877396e+17,3560.000000,40.000000,3.000000,171.000000,158.000000,186.195496,20.584429,2.819681,7.470993,...,0.015726,0.014031,0.014812,0.026565,4.006657,4.000000,35.000000,46.000000,5.000000,23.000000
75%,5.880155e+17,4512.000000,40.000000,5.000000,289.000000,231.000000,226.771229,38.181531,3.804628,9.785356,...,0.020539,0.017795,0.018936,0.037277,5.535648,6.000000,51.000000,62.000000,9.000000,52.000000
max,8.647475e+18,6793.000000,44.000000,6.000000,862.000000,2949.000000,359.995021,76.853188,106.211490,218.497340,...,99999.000000,99999.000000,99999.000000,99999.000000,104.335280,8.000000,999.000000,999.000000,999.000000,121.000000


In [63]:
metadata.shape

(325704, 63)

In [64]:
is_null_md = metadata.isnull().sum()

null_columns = is_null_md[is_null_md>0]
print(null_columns)

REDSHIFT          53162
REDSHIFTERR       53162
PETROMAG_MU       53163
PETROMAG_MG       53162
PETROMAG_MR       53162
PETROMAG_MI       53162
PETROMAG_MZ       53162
PETROMAGERR_MU    53162
PETROMAGERR_MG    53162
PETROMAGERR_MR    53162
PETROMAGERR_MI    53162
PETROMAGERR_MZ    53162
PETROR50_R_KPC    53162
dtype: int64


In [65]:
metadata_before = metadata.copy()
metadata = metadata.dropna()
rows_dropped = metadata_before.shape[0]-metadata.shape[0]

print("Metadata dataset shape before: ", metadata_before.shape)
print("Metadata dataset shape after: ", metadata.shape)

print("Rows dropped: ", rows_dropped)

Metadata dataset shape before:  (325704, 63)
Metadata dataset shape after:  (272541, 63)
Rows dropped:  53163


In [66]:
dataset = metadata.copy()
dataset["target"] = target_col

print(dataset.columns.tolist())

['OBJID', 'RUN', 'RERUN', 'CAMCOL', 'FIELD', 'OBJ', 'RA', 'DEC', 'PETROR50_R', 'PETROR90_R', 'PETROMAG_U', 'PETROMAG_G', 'PETROMAG_R', 'PETROMAG_I', 'PETROMAG_Z', 'PETROMAGERR_U', 'PETROMAGERR_G', 'PETROMAGERR_R', 'PETROMAGERR_I', 'PETROMAGERR_Z', 'PSFMAG_R', 'FIBERMAG_R', 'DEVMAG_R', 'DEVMAGERR_R', 'EXPMAG_R', 'EXPMAGERR_R', 'FRACDEV_R', 'MU50_R', 'EXTINCTION_U', 'EXTINCTION_G', 'EXTINCTION_R', 'EXTINCTION_I', 'EXTINCTION_Z', 'ROWC_U', 'COLC_U', 'ROWC_G', 'COLC_G', 'ROWC_R', 'COLC_R', 'ROWC_I', 'COLC_I', 'ROWC_Z', 'COLC_Z', 'CMODELMAG_R', 'CMODELMAGERR_R', 'REDSHIFT', 'REDSHIFTERR', 'PETROMAG_MU', 'PETROMAG_MG', 'PETROMAG_MR', 'PETROMAG_MI', 'PETROMAG_MZ', 'PETROMAGERR_MU', 'PETROMAGERR_MG', 'PETROMAGERR_MR', 'PETROMAGERR_MI', 'PETROMAGERR_MZ', 'PETROR50_R_KPC', 'REGION', 'PETROR50_R_KPC_SIMPLE_BIN', 'PETROMAG_MR_SIMPLE_BIN', 'REDSHIFT_SIMPLE_BIN', 'WVT_BIN', 'target']
